Project Goals: 1- Analyzing user behavior to extract actionable insights.
2- Generate automated reports in Excel/PDF.
3- Creating a multi-level interactive dashboard.
4- Data simulation, cleaning, merging, enrichment.
5- Insights and recommendations.

In [1]:
import pandas as pd
import numpy as np
np.random.seed(50)
dates = pd.date_range(start = '2000-01-01', end='2025-12-31', freq = 'ME')
user_ids = [f"user_{i}" for i in range(1,250)]
content_ids = [f"content_{i}" for i in range(1,50)]
devices = ["Laptop", "Desktop computer", "Tablet", "Smartphone"]
data = {
    "timestamp": np.random.choice(dates, 2000),
    "user_id": np.random.choice(user_ids, 2000),
    "content_id": np.random.choice(content_ids, 2000),
    "session_duration": np.random.randint(30, 600, 2000),
    "device": np.random.choice(devices, 2000)
}
df = pd.DataFrame(data)
df.to_csv("data/user_data.csv", index=False)


Content data 

In [2]:
content = {"content_id": [f"cont_{i}" for i in range(1,21)],
          "category": np.random.choice(['news','tutorial','reviews'],20),
           # "author": [f"author_{"i%5}" for in in range(1,21)]
            "author": [f"author_{i%5}" for i in range(1,21)]
}
df_content = pd.DataFrame(content)
df_content.to_csv("data/content_meta.csv", index=False)

In [3]:
def load_clean():
    df = pd.read_csv(r"data/user_data.csv", parse_dates = ["timestamp"])
    df.dropna(inplace = True)
    df['Day'] = df['timestamp'].dt.day_name()
    df['Month'] = df['timestamp']. dt.month
    df_content = pd.read_csv(r"data/content_meta.csv")
    df = df.merge(df_content, on="content_id", how='left')

    #df.merge(df_content, on ="content_id", how = 'left')
    return df
if __name__=="__main__": #Checks if the file is being run directly, not imported.Only the code inside this block will execute when the file is run directly.
    df_new = load_clean()
    print(df_new.head())

   timestamp   user_id  content_id  session_duration            device  \
0 2014-09-30  user_248  content_48               187            Tablet   
1 2009-02-28  user_161  content_15               554        Smartphone   
2 2024-02-29  user_134  content_48                68        Smartphone   
3 2011-01-31   user_30  content_39               203            Laptop   
4 2005-11-30  user_181   content_2               471  Desktop computer   

         Day  Month category author  
0    Tuesday      9      NaN    NaN  
1   Saturday      2      NaN    NaN  
2   Thursday      2      NaN    NaN  
3     Monday      1      NaN    NaN  
4  Wednesday     11      NaN    NaN  


Compute KPI's

In [53]:
def kpi(df):
    kpis={}
    kpis['daily_active_user'] =df.groupby(df['timestamp'].dt.date)['user_id'].unique()
    #kpis['ave_session_duration ']= df.groupby(df['timestamp'].dt.date)['session_duaration'].mean()
    kpis['ave_session_duration'] = df.groupby(df['timestamp'].dt.date)['session_duration'].mean()

    kpis['most_viewed_content'] = df['content_id'].value_counts().head(5)
    kpis['device_distribution'] =df['device'].value_counts(normalize = True)
    return kpis
if __name__=="__main__":
    #from data_pipeline import load_clean
    df = load_clean()
    kpis = kpi(df)
    print(kpis)

{'daily_active_user': timestamp
2000-01-31    [user_41, user_25, user_31, user_55, user_9, u...
2000-02-29               [user_222, user_138, user_2, user_215]
2000-03-31    [user_25, user_4, user_136, user_86, user_106,...
2000-04-30    [user_53, user_136, user_25, user_22, user_194...
2000-05-31    [user_213, user_207, user_93, user_153, user_2...
                                    ...                        
2025-08-31    [user_185, user_81, user_158, user_168, user_1...
2025-09-30    [user_138, user_32, user_179, user_229, user_1...
2025-10-31    [user_61, user_91, user_44, user_212, user_163...
2025-11-30    [user_146, user_24, user_52, user_73, user_200...
2025-12-31    [user_174, user_55, user_240, user_201, user_112]
Name: user_id, Length: 311, dtype: object, 'ave_session_duration': timestamp
2000-01-31    355.714286
2000-02-29    248.250000
2000-03-31    239.333333
2000-04-30    257.714286
2000-05-31    363.125000
                 ...    
2025-08-31    441.666667
2025-09-30  

Dashboard:

Predicting Trend

In [4]:
from sklearn.linear_model import LinearRegression
def predict_dau(df):
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    #daily_active_users = df.groupby(df['timestamp'].dt.date)['user_id'].nunique().reset_index(name='users')
    #daily_active_users['day_number'] = (daily_active_users['timestamp':'date'] - daily_active_users['timestamp'].min()).dt.days
    daily_active_users = df.groupby(df['timestamp'].dt.date)['user_id'].nunique().reset_index(name='users')
    daily_active_users.rename(columns={'timestamp':'date'}, inplace=True)
    daily_active_users['date'] = pd.to_datetime(daily_active_users['date'])
    daily_active_users['day_number'] = (daily_active_users['date'] - daily_active_users['date'].min()).dt.days
    X = daily_active_users[['day_number']]
    y = daily_active_users['users']
    
    model = LinearRegression()
    model.fit(X, y)
    
    future_days = np.arange(daily_active_users['day_number'].max() + 1, daily_active_users['day_number'].max() + 8).reshape(-1,1)
    predictions = model.predict(future_days)
    return predictions

if __name__ == "__main__":
    #from data_pipeline import load_and_clean
    df = load_clean()
    predicted_dau = predict_dau(df)
    print("Next 7 days DAU prediction:", predicted_dau)

Next 7 days DAU prediction: [6.44538698 6.44540843 6.44542988 6.44545133 6.44547278 6.44549423
 6.44551568]


C:\Users\User\anaconda3\envs\macro_analysis\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


What We Achieved: 1- Retention & cohort analysis to understand user engagement over time. 2- Trend prediction to anticipate user activity.